In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
# Define transforms for data augmentation
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the datasets
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/AnimalClassification model/Train', transform=train_transforms)
valid_dataset = datasets.ImageFolder('/content/drive/MyDrive/AnimalClassification model/Test', transform=test_transforms)

# Create the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True)

# Define the classes

classes = os.listdir("/content/drive/MyDrive/AnimalClassification model/Train")


In [ ]:
print(classes)

['Deer', 'Elephant', 'Fox', 'Horse', 'Leopard', 'Lion', 'Monkey', 'Otter', 'Rabbit', 'Tiger', 'Wolf', 'Cheetah']


In [ ]:
class AnimalCNN(nn.Module):
    def __init__(self):
        super(AnimalCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512,12)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = AnimalCNN()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_losses = []
train_accuracies = []

In [ ]:
num_epochs = 100

train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

best_valid_acc = 0.0  # Track the best validation accuracy so far

for epoch in range(num_epochs):
    running_train_loss = 0.0
    running_train_corrects = 0
    model.train()
    if((epoch+1)%10==0):
      PATH=f"/content/drive/MyDrive/AnimalClassification model/ACCNN{epoch+1}.pth"
      torch.save(model.state_dict(), PATH)
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs.data, 1)
        running_train_loss += loss.item() * images.size(0)
        running_train_corrects += torch.sum(preds == labels.data)
        
    epoch_train_loss = running_train_loss / len(train_dataset)
    epoch_train_acc = running_train_corrects / len(train_dataset)
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_acc)
    
    # Evaluate on validation set
    model.eval()
    with torch.no_grad():
        running_valid_loss = 0.0
        running_valid_corrects = 0
        for images, labels in valid_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs.data, 1)
            running_valid_loss += loss.item() * images.size(0)
            running_valid_corrects += torch.sum(preds == labels.data)
            
        epoch_valid_loss = running_valid_loss / len(valid_dataset)
        epoch_valid_acc = running_valid_corrects / len(valid_dataset)
        valid_losses.append(epoch_valid_loss)
        valid_accuracies.append(epoch_valid_acc)
    
    # Check if this is the best validation accuracy so far and save the model
    if epoch_valid_acc > best_valid_acc:
        best_valid_acc = epoch_valid_acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/AnimalClassification model/best_model.pth")
    
  # Print the epoch statistics
    print(f"Epoch {epoch+1} Train Loss: {epoch_train_loss:.4f} Train Acc: {epoch_train_acc:.4f} \
       Valid Loss: {epoch_valid_loss:.4f} Valid Acc: {epoch_valid_acc:.4f}")



100%|██████████| 188/188 [1:13:43<00:00, 23.53s/it]


Epoch 1 Train Loss: 1.8951 Train Acc: 0.3287        Valid Loss: 1.4847 Valid Acc: 0.5033


100%|██████████| 188/188 [05:35<00:00,  1.78s/it]


Epoch 2 Train Loss: 1.2087 Train Acc: 0.5721        Valid Loss: 1.3353 Valid Acc: 0.5708


100%|██████████| 188/188 [05:35<00:00,  1.78s/it]


...

100%|██████████| 188/188 [06:16<00:00,  2.00s/it]


Epoch 21 Train Loss: 0.0510 Train Acc: 0.9852        Valid Loss: 2.2772 Valid Acc: 0.6933


100%|██████████| 188/188 [06:12<00:00,  1.98s/it]


Epoch 22 Train Loss: 0.0996 Train Acc: 0.9710        Valid Loss: 2.2256 Valid Acc: 0.7042


 79%|███████▊  | 148/188 [04:57<01:25,  2.13s/it]